In [ ]:
#基础数据
import requests
import pandas as  pd
import json
proxy = {
    "http": "http://127.0.0.1:10808",
    "https": "http://127.0.0.1:10808",
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
url = 'https://studio-api.prod.suno.com/api/discover'
session = requests.Session()

In [ ]:
#请求函数
import requests
import json

from typing import Optional, Dict, Any

def make_suno_discover_request(body: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    url = "https://studio-api.prod.suno.com/api/discover"
    headers = {
        "accept": "*/*",
        "accept-language": "zh-CN,zh;q=0.9",
        "affiliate-id": "undefined",
        "authorization": "Bearer null",
        # "browser-token": "{\"token\":\"eyJ0aW1lc3RhbXAiOjE3NDk1NjM4NTE3Nzl9\"}",
        "cache-control": "no-cache",
        "content-type": "text/plain;charset=UTF-8",
        # "device-id": "94f64167-9cb1-4c77-99f0-33035be655f1",
        "pragma": "no-cache",
        "priority": "u=1, i",
        "sec-ch-ua": "\"Google Chrome\";v=\"137\", \"Chromium\";v=\"137\", \"Not/A)Brand\";v=\"24\"",
        "sec-ch-ua-mobile": "?1",
        "sec-ch-ua-platform": "\"Android\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site"
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(body),proxies=proxy)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"请求发生错误: {e}")
        return None

In [ ]:
#列表查重
import pandas as pd
import os
from typing import List, Dict, Any

def update_suno_tenders_csv(new_data_list: List[Dict[str, Any]], csv_file_path: str) -> None:
    expected_columns = ['title', 'video_url']

    # 确保CSV文件存在，如果不存在则创建一个空的
    if not os.path.exists(csv_file_path):
        if new_data_list:
            # 明确指定列名
            initial_df = pd.DataFrame(columns=expected_columns)
            initial_df.to_csv(csv_file_path, index=False)
        else:
            print("新数据列表为空，无法创建带有列名的CSV文件。请手动创建或提供至少一条数据。")
            return

    # 读取现有的CSV文件
    # 读取现有的CSV文件
    try:
        existing_df = pd.read_csv(csv_file_path, header=None, names=expected_columns)
    except pd.errors.EmptyDataError:
        existing_df = pd.DataFrame(columns=expected_columns) # 如果文件为空，则创建一个空的DataFrame，并指定列名
    except Exception as e:
        print(f"读取CSV文件时发生错误: {e}")
        existing_df = pd.DataFrame(columns=expected_columns)
    # 将新数据转换为DataFrame
    new_df = pd.DataFrame(new_data_list)

    # 确保新数据DataFrame包含所有预期列，并按顺序排列
    # 如果new_df中缺少video_url，则添加一个空字符串列
    for col in expected_columns:
        if col not in new_df.columns:
            new_df[col] = ''
    new_df = new_df[expected_columns] # 重新排序以匹配预期

    # 检查新数据中是否有重复的title与现有数据重复
    if not existing_df.empty:
        existing_titles = set(existing_df['title'].astype(str))
        # 过滤掉新数据中title已存在于现有数据中的条目
        filtered_new_df = new_df[~new_df['title'].astype(str).isin(existing_titles)]
    else:
        filtered_new_df = new_df

    # 将过滤后的新数据追加到CSV文件
    if not filtered_new_df.empty:
        filtered_new_df.to_csv(csv_file_path, mode='a', header=False, index=False)
        print(f"成功将 {len(filtered_new_df)} 条新数据追加到 {csv_file_path}")
    else:
        print("没有新的不重复数据需要追加。")

In [ ]:
#下载函数
import yt_dlp
import os
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def hook(self, d):
        if d['status'] == 'downloading':
            if self.total is None:
                self.total = d.get('total_bytes') or d.get('total_bytes_estimate', 0)
            self.update(d['downloaded_bytes'] - self.n)
        elif d['status'] == 'finished':
            self.close()

def download_videos(download_path: str, url_list: list):
    """
    下载给定列表中的视频链接，支持断点续传，跳过已存在视频，实时显示下载进度，并使用代理。

    Args:
        download_path (str): 视频下载保存的目录。
        url_list (list): 包含视频URL的列表。
    """
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"创建下载目录: {download_path}")

    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'noplaylist': True,  # 不下载播放列表
        'continue': True,    # 启用断点续传
        'download_archive': os.path.join(download_path, 'downloaded_videos.txt'), # 记录已下载的视频，用于跳过
        'progress_hooks': [DownloadProgressBar(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc="下载进度").hook],
        'proxy': 'http://127.0.0.1:10808', # 使用本地10808端口代理
        'ignoreerrors': True, # 忽略下载错误，继续下载下一个
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for url in url_list:
            print(f"\n开始下载: {url}")
            try:
                ydl.download([url])
            except Exception as e:
                print(f"下载 {url} 时发生错误: {e}")
        print("\n所有视频下载完成。")



In [ ]:
# 每日热门
body: Dict[str, Any] = {
    "start_index": 1,
    "page_size": 1,
    "section_name": None,
    "section_content": None,
    "secondary_section_content": None,
    "product": None
}
data = make_suno_discover_request(body)
songs = data['sections'][0]['items'][2]['items']
song_data = pd.DataFrame({
    'title': [item['title'] for item in songs],
    'video_url': [item['video_url'] for item in songs],
    })

print(song_data.to_dict(orient='records'))
update_suno_tenders_csv(song_data.to_dict(orient='records'), 'suno_tenders.csv')

In [ ]:
# 每周热门
body:Dict[str,Any]={
  "start_index":0,
  "page_size":1,
  "section_name":"trending_songs",
  "section_content":"Global",
  "secondary_section_content":"Weekly",
  "page":1,
  "section_size":20,
  "disable_shuffle":False
}
data = make_suno_discover_request(body)
songs = data['sections'][0]['items']
song_data = pd.DataFrame({
    'title': [item['title'] for item in songs],
    'video_url': [item['video_url'] for item in songs],
    })

print(song_data.to_dict(orient='records'))
update_suno_tenders_csv(song_data.to_dict(orient='records'), 'suno_tenders.csv')


In [ ]:
# 每月热门
body:Dict[str,Any]={
  "start_index":0,
  "page_size":1,
  "section_name":"trending_songs",
  "section_content":"Global",
  "secondary_section_content":"Monthly",
  "page":1,
  "section_size":20,
  "disable_shuffle":False
}
data = make_suno_discover_request(body)
songs = data['sections'][0]['items']
song_data = pd.DataFrame({
    'title': [item['title'] for item in songs],
    'video_url': [item['video_url'] for item in songs],
    })
print(song_data.to_dict(orient='records'))
update_suno_tenders_csv(song_data.to_dict(orient='records'), 'suno_tenders.csv')

In [ ]:
# 总热门
body:Dict[str,Any]={
    "start_index":0,"page_size":1,
    "section_name":"trending_songs",
    "section_content":"Global",
    "secondary_section_content":"All Time",
    "page":1,
    "section_size":20,
    "disable_shuffle":False
}
data = make_suno_discover_request(body)
songs = data['sections'][0]['items']
song_data = pd.DataFrame({
    'title': [item['title'] for item in songs],
    'video_url': [item['video_url'] for item in songs],
    })

print(song_data.to_dict(orient='records'))
update_suno_tenders_csv(song_data.to_dict(orient='records'), 'suno_tenders.csv')


In [ ]:
#v4.5
body: Dict[str, Any] ={
    "start_index": 2,
    "page_size": 1,
    "section_name": None,
    "section_content": None,
    "secondary_section_content": None,
    "product": None
}
data = make_suno_discover_request(body)
songs=data['sections'][0]['items']
song_data = pd.DataFrame({
    'title': [item['title'] for item in songs],
    'video_url': [item['video_url'] for item in songs],
})
print(song_data.to_dict(orient='records'))
update_suno_tenders_csv(song_data.to_dict(orient='records'), 'suno_tenders.csv')


In [ ]:
# 下载视频
songs = pd.read_csv('suno_tenders.csv', header=None, names=['title', 'video_url']).to_dict(orient='records')
song_urls = [song['video_url'] for song in songs]
download_path = 'suno_videos'  # 视频下载保存的目录
# 下载视频
download_path = '/media/ttuubb/1C49-4CEA/sunoai'  # 视频下载保存的目录
download_videos(download_path, song_urls)